# TP n°1 : Exploration de pré-traitement de données

Dans ce TP, nous allons voir :
- comment inspecter et visualiser le contenu d'un jeu de données avec les bibliothèques `pandas` et `seaborn`,
- comment entraîner un modèle scikit-learn et mesurer sa performance en séparant données d'entraînement et données de test,
- comment pré-traîter les données numériques et catégoriques pour augmenter la performance d'un modèle.

Votre travail est de compléter les cellules de code qui contiennent un commentaire "# A compléter ici".
Commencez par remplir la case ci-dessous.

In [ ]:
# A compléter ici :
# NOM :
# Prénom :
# N° étudiant :

## Documentation :

**Pendant le TP, n'hésitez pas à aller consulter la [documentation de scikit-learn](https://scikit-learn.org/stable/api/index.html), accessible à ce lien : [https://scikit-learn.org/stable/api/index.html](https://scikit-learn.org/stable/api/index.html)**. Elle contient de nombreuses explications sur comment utiliser les fonctions introduites dans ce TP.

## 0. Préparation du TP

Le code ci-dessous installe puis importe les librairies python nécessaire pour le TP.

In [ ]:
!pip install numpy matplotlib pandas scikit-learn seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

## 1. Exploration et visualisation de données

On commence par charger les données du dataset "[palmer penguins](https://allisonhorst.github.io/palmerpenguins/)". L'objet qu'on obtient est une [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) pandas, un type spécialisé pour la gestion de jeux de données.

La fonction `.info()` d'une DataFrame permet d'afficher des informations sur le jeu de données.

In [ ]:
data_penguins = sns.load_dataset("penguins")
data_penguins.info()

La fonction `.head()` d'une DataFrame permet d'afficher le nom de ses colonnes ainsi que le contenu de ses 5 premières lignes.

In [ ]:
data_penguins.head()

Pour accéder à une colonne spécifique de la dataframe, on écrit `dataframe["nom_colonne"]`.
Ensuite, la fonction `value_counts()` renvoie des informations sur les valeurs dans la colonne : les différentes valeurs et leur fréquence.


In [ ]:
species = data_penguins["species"]
species.value_counts()

On peut observer ici que le jeu de données contient 68 pingouins de l'espèce "Chinstrap".

Pour obtenir des informations sur les données numériques, on peut utiliser la fonction `.describe()`, comme suit:

In [ ]:
data_penguins.describe()

**Q° 1**: écrire du code permettant de savoir combien de pingouins femelle le jeu de données contient.

In [ ]:
# A compléter ici

La fonction `.hist()` permet d'afficher un histogramme pour visualiser la répartition des données numériques.

In [ ]:
_ = data_penguins.hist()

La fonction `seaborn.pairplot()` permet d'afficher des graphiques montrant la répartition d'une colonne en fonction d'une autre. Le paramètre `hue` permet de choisir une colonne selon laquelle colorier les points. Ici, on choisit la colonne qu'on essaiera plus tard de prédire, l'espèce.

In [ ]:
_ = sns.pairplot(data_penguins, hue="species")

### Exercice : à vous !

1. Chargez le dataset "iris" avec la fonction `load_dataset()` de `seaborn`

In [ ]:
# A compléter ici

2. Combien de colonnes catégoriques ? Combien sont numériques ?

In [ ]:
# A compléter ici

3. Affichez un histogramme de chaque feature (colonne) numérique

In [ ]:
# A compléter ici

4. Affichez la distribution de chaque paire de features, en fonction de l'espèce d'iris (colonne "species").

In [ ]:
# A compléter ici

5. Au vu de ces graphes, pensez vous qu'il soit facile de déterminer l'espèce d'une iris à partir des colonnes `petal_width` et  `petal_length` ?

> Répondre ici

# 2. Entraîner un modèle sur des données numériques

On va dans un premier temps se restreindre aux données numériques dans le jeu de données "palmer penguins", en ignorant les lignes qui contiennent des valeurs manquantes.

In [ ]:
data_penguins_no_na = data_penguins.dropna()
numerical_columns = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]
data = data_penguins_no_na[numerical_columns]
target = data_penguins_no_na["species"]

La première étape est de séparer les données deux, une partie pour l'entraînement et une partie pour la validation.  
Ici, test size indique la proportion des données à mettre dans la partie de validation; ici, 0.3 signifie 30%.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state=0)

On entraîne un modèle linéaire simple, la régression logistique (que l'on discutera plus en détail dans un cours à venir), sur les données d'entraînement.

In [ ]:
from sklearn.linear_model import LogisticRegression
from time import time

model = LogisticRegression()
start = time()
_ = model.fit(X_train, y_train)
elapsed = time() - start

On calcule maintenant sa précision sur le jeu de données de test. Vous devriez observer un score d'environ 97% en environ 0.2 sec.

In [ ]:
score = model.score(X_test, y_test)
print(f"Score sur les données de test: {score*100:.3f}%, entraîné en {elapsed:.3f}s.")

## Amélioration des performances : scaling des données.

Les modèles linéaires sont sensibles à l'ordre de grandeur des données reçus en entrée, et fonctionnent mieux lorsque les données sont centrées et réduites.
Ce n'est pas le cas de nos données, c'est pourquoi le modèle linéaire emet un `ConvergenceWarning` lors de l'appel à `fit()`.

Pour essayer d'améliorer les performances du modèle, on va centrer et réduire les données à l'aide d'un `StandardScaler` avant de les passer au modèle. 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

scaler = StandardScaler()
classifier = LogisticRegression()
scaled_model = make_pipeline(scaler, classifier)

start = time()
scaled_model.fit(X_train, y_train)
elapsed = time() - start

In [ ]:
score = scaled_model.score(X_test, y_test)
print(f"Score sur les données de test: {score*100:.3f}%, entraîné en {elapsed:.3f}s.")

On observe qu'après avoir scalé les données, la précision du modèle est restée la même, mais le temps d'entraînement a diminué !

## Exercice : à vous !

Dans cet exerice, on utilisera un jeu de données contenant des informations sur des quarties en Californie, à partir desquelles il faut prédire le prix vente median d'une maison dans le quartier ("MedHouseVal"). 

In [ ]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True).frame
target = california_housing["MedHouseVal"]
data = california_housing.drop(columns = ["MedHouseVal"])

california_housing.head()

1. La colonne cible ("MedHouseVal") est-elle numérique ou catégorique ? En conséquence, s'agit-il d'un problème de régression ou de classification ?

> Répondre ici

2. Séparer les données en un ensemble d'entraînement contenant 80% des données et un ensemble de test contenant 20% des données.

In [ ]:
# A compléter ici

3. Entraîner un modèle linéaire adapté au type de problème (`LinearRegression` pour la régression, `LogisticRegression` pour la classification), en mesurant le temps d'entraînement.
Quelle est la précision du modèle sur les données de test ? Et sur les données d'entraînement ? Est-ce normal ?

In [ ]:
# A compléter ici

# 3. Travailler avec les données catégoriques

Dans cette dernière partie du TP, nous allons entraîner un modèle de classification sur le jeu de données "Adult census", que l'on a vu en cours Mardi.
Le but sera d'utiliser à la fois les données numériques et les données catégoriques dans le modèle.

In [ ]:
!curl -L -o adult-census-income.zip https://www.kaggle.com/api/v1/datasets/download/uciml/adult-census-income
!unzip -o adult-census-income.zip

In [ ]:
# data (as pandas dataframes) 
data = pd.read_csv("adult.csv").dropna()
X = data[["age", "workclass", "fnlwgt", "education", "marital.status", "occupation", "relationship", "race", "sex", "capital.gain", "capital.loss", "hours.per.week", "native.country"]]
y = data["income"]

In [ ]:
X.head()

Plutôt que d'entrer à la main le nom des colonnes numériques et catégoriques, on va utiliser la fonction `make_column_selector` de `scikit-learn` pour les détecter automatiquement.
Les paramètres `dtype_exclude` et `dtype_include` indique quel type de colonne on veut enlever où garder. Ici, les colonnes catégoriques sont encodées par des `string`, on peut donc les isoler en passant le paramètre `object`. Les colonnes numériques seront généralement du type `float64` ou `int32`. 

In [ ]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)
numerical_columns, categorical_columns

On crée ensuite les objets qui transformeront chaque ensemble de colonne. Ici, on encodera les données catégoriques avec un `OrdinalEncoder`, qui assigne un entier différent à chaque classe. Les données numériques seront centrées et réduites avec un `StandardScaler`, qui met leur moyenne à 0 et leur écart-type à 1.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
numerical_preprocessor = StandardScaler()

Ensuite, on associe chaque objet à son ensemble de colonnes dans un `ColumnTransformer`.

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
        ("standard_scaler", numerical_preprocessor, numerical_columns),
    ]
)

On peut alors créer un modèle linéaire de classification, en utilisant `LogisticRegression`, et on le combine avec notre pré-processeur en utilisant `make_pipeline`.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

Enfin, on peut entraîner notre modèle et mesurer sa performance.

Dans les sections précédentes, on avait utilisé la fonction `train_test_split` pour séparer les données en un jeu d'entraînement et un jeu de test. Une limitation de cette méthode est qu'elle ne permet de faire qu'une seule mesure empirique de la précision, et ne donne pas d'information sur sa variabilité : il est possible que le modèle soit performant "par chance".

On va utiliser ici la cross validation, qui entraîne automatiquement le modèle sur plusieurs séparations en deux du jeu de données, et renvoie le score pour chaque itération. Cela permet de mesurer la sensibilité de la performance au choix du jeu de données d'entraînement. 

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5, scoring="balanced_accuracy")

In [ ]:
print(f"Score: {100*np.mean(scores):.3f}% +/- {100*np.std(scores):.2f}%")

Ici, le paramètre `scoring="balanced_accuracy"` signifie que l'on veut compenser le déséquilibre des valeurs dans `y`. En effet, la valeur "<=50K" apparaît 24 720 fois, alors que la valeur "<50k" apparaît seulement 7841 fois! Un modèle qui répondrait tout le temps "<=50K" aurait donc une précision de plus de 75% !

In [ ]:
y.value_counts()

## Exercice : à vous !

Dans la section précédente, on a utilisé un `OrdinalEncoder` pour encoder les variables catégoriques. Les modèles linéaires ont du mal àutiliser les données encodées en ordinaux.
Dans cet exercice, on va entrainer le même modèle, mais en encodant les colonnes catégoriques avec un `OneHotEncoder`, et regarder si on observe une différence de performance ou non.

1. Créez un `OneHotEncoder` pour encoder les colonnes catégoriques et un `StandardScaler` pour encoder les variables numériques.

In [ ]:
# A compléter ici

2. Créez un `ColumnTransformer` qui prend en charge la transformation de toutes les colonnes, en utilisant les objets de la question précédente.

In [ ]:
# A compléter ici

3. Créez un modèle de classification basé sur une `LogisticRegression` avec paramètre `max_iter=500`, et le préprocesseur de la question précédente.

In [ ]:
# A compléter ici

4. Utilisez `cross_val_score` avec `cv=5` et `scoring="balanced_accuracy"` pour mesurer les performances du nouveau modèle. Sont-elles meilleures que pour celui où les données catégoriques sont encodées avec un `OrdinalEncoder` ?
Cela correspond-t-il à ce que l'on attendait ?

In [ ]:
# A compléter ici

## Exercice : à vous !

On a jusqu'ici considéré les modèles linéaires (`LinearRegression` et `LogisticRegression`). Dans cet exercice, on s'intéressera à un modèle plus complexe, `sklearn.ensemble.HistGradientBoostingClassifier`.

Le but est de répondre aux questions suivantes :
- Le scaling des colonnes améliore-t-il les performances d'un `HistGradientBoostingClassifier` ?
- Quel est le meilleure encodage de featues catégoriques pour ce modèle : one-hot ou ordinal ?

1. Créez un modèle qui encode les colonnes catégoriques en **ordinaux** et ne change pas les colonnes numériques (vous pouvez utiliser l'argument `remainder="passthrough"` de `ColumnTransformer`). Mesurez sa performance à l'aide de la cross-validation.

In [ ]:
# A compléter ici

2. Créez un modèle qui encode les colonnes catégoriques en ordinaux et **applique du scaling** aux colonnes numériques. Mesurez sa performance à l'aide de la cross-validation.

In [ ]:
# A compléter ici

3. Que pouvez-vous en conclure sur l'influence du scaling sur les performances du `HistGradientBoostingClassifier` ?

In [ ]:
# A compléter ici

4. Créez maintenant un modèle qui encode les colonnes catégoriques en **one-hot**, et **applique du scaling** aux colonnes numériques. Mesurez sa performance à l'aide de la cross-validation.

In [ ]:
# A compléter ici

5. Que pouvez-vous en conclure ?

In [ ]:
# A compléter ici